In [1]:
from tia.bbg import LocalTerminal
import pandas as pd
import numpy as np

# Index Members

In [2]:
index = 'SPX Index'
field = 'INDX_MEMBERS'
resp = LocalTerminal.get_reference_data(index, field, DISPLAY_ID_BB_GLOBAL_OVERRIDE = 'Y')

In [3]:
members = resp.as_map()[index][field]
members.columns = ['id']

In [4]:
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 1 columns):
id    505 non-null object
dtypes: object(1)
memory usage: 4.0+ KB


# Historical Excess returns

In [5]:
'/bbgid/' + members.head()

,id
0,/bbgid/BBG000C2V541
1,/bbgid/BBG005P7Q8V5
2,/bbgid/BBG000F7RFH6
3,/bbgid/BBG000B9Y5X2
4,/bbgid/BBG003P7K0B5


In [6]:
from scipy.stats.mstats import gmean
test_securities = '/bbgid/' + members['id'].head(10)
returns = LocalTerminal.get_historical(test_securities, 
                                       ['DAY_TO_DAY_TOT_RETURN_GROSS_DVDS'], start='1/1/2014', end='1/1/2016').as_frame() / 100
returns.columns = returns.columns.droplevel(1)
returns.dropna(axis = 1, inplace = True)
returns = returns + 1
returns = returns.rolling(255, min_periods=255).apply(gmean).shift(-255).dropna() - 1
returns.head()

,/bbgid/BBG005P7Q8V5,/bbgid/BBG003P7K0B5,/bbgid/BBG000BB6RX0,/bbgid/BBG000MDCSN6,/bbgid/BBG000B9ZZG4,/bbgid/BBG000C2V541,/bbgid/BBG000B9Y5X2,/bbgid/BBG000F7RFH6,/bbgid/BBG000BB5BP5,/bbgid/BBG000D9DC22
date,,,,,,,,,,
2014-01-02,0.002915,0.001118,0.000455,0.001130,0.000705,-0.000011,0.001299,0.001480,0.000713,0.000441
2014-01-03,0.002785,0.001135,0.000498,0.001136,0.000743,0.000055,0.001534,0.001403,0.000820,0.000487
2014-01-06,0.002591,0.001172,0.000517,0.001184,0.000650,0.000046,0.001517,0.001421,0.000831,0.000525
2014-01-07,0.002412,0.001163,0.000457,0.001053,0.000714,-0.000057,0.001446,0.001339,0.000735,0.000465
2014-01-08,0.002380,0.001059,0.000402,0.001046,0.000615,-0.000176,0.001457,0.001240,0.000710,0.000449


In [7]:
risk_free = LocalTerminal.get_historical('/bbgid/BBG002SCTH72', ['PX_LAST'], start='1/1/2014', end='1/1/2016').as_frame()
risk_free.columns = risk_free.columns.droplevel(1)
risk_free.head()

,/bbgid/BBG002SCTH72
date,
2014-01-01,0.04660
2014-01-02,0.04854
2014-01-03,0.04578
2014-01-06,0.04154
2014-01-07,0.03659


In [8]:
excess_returns = returns.subtract(risk_free['/bbgid/BBG002SCTH72'], axis = 0).dropna()
excess_returns.head()

,/bbgid/BBG005P7Q8V5,/bbgid/BBG003P7K0B5,/bbgid/BBG000BB6RX0,/bbgid/BBG000MDCSN6,/bbgid/BBG000B9ZZG4,/bbgid/BBG000C2V541,/bbgid/BBG000B9Y5X2,/bbgid/BBG000F7RFH6,/bbgid/BBG000BB5BP5,/bbgid/BBG000D9DC22
date,,,,,,,,,,
2014-01-02,-0.045625,-0.047422,-0.048085,-0.047410,-0.047835,-0.048551,-0.047241,-0.047060,-0.047827,-0.048099
2014-01-03,-0.042995,-0.044645,-0.045282,-0.044644,-0.045037,-0.045725,-0.044246,-0.044377,-0.044960,-0.045293
2014-01-06,-0.038949,-0.040368,-0.041023,-0.040356,-0.040890,-0.041494,-0.040023,-0.040119,-0.040709,-0.041015
2014-01-07,-0.034178,-0.035427,-0.036133,-0.035537,-0.035876,-0.036647,-0.035144,-0.035251,-0.035855,-0.036125
2014-01-08,-0.033790,-0.035111,-0.035768,-0.035124,-0.035555,-0.036346,-0.034713,-0.034930,-0.035460,-0.035721


# Factor Exposures

In [9]:
# Earnings to Price Risk Premium
E2P = LocalTerminal.get_reference_data(test_securities, ['PE_RATIO'], FUND_PER = '2014').as_frame()**-1
E2P = (E2P - E2P.mean()) / E2P.std()
E2P.describe()

C:\Users\nschantz\AppData\Local\conda\conda\envs\test_env\lib\site-packages\tia\bbg\v3api.py:409: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  frame = frame.reindex_axis(self.request.fields, axis=1)


,PE_RATIO
count,1.000000e+01
mean,-4.440892e-17
std,1.000000e+00
min,-1.401649e+00
25%,-6.950736e-01
50%,-2.278762e-02
75%,3.873251e-01
max,2.053602e+00


# Characteristic Portfolio Holdings

In [15]:
np.random.seed(42)
weights = np.random.sample(10)
weights = weights / weights.sum()
np.sum(weights)

1.0

### Asset Characteristics

In [23]:
E2P['PE_RATIO'] * weights

/bbgid/BBG000B9Y5X2    0.033185
/bbgid/BBG000B9ZZG4   -0.181529
/bbgid/BBG000BB5BP5   -0.197256
/bbgid/BBG000BB6RX0   -0.032735
/bbgid/BBG000C2V541   -0.024955
/bbgid/BBG000D9DC22   -0.005467
/bbgid/BBG000F7RFH6    0.001526
/bbgid/BBG000MDCSN6    0.145806
/bbgid/BBG003P7K0B5    0.019268
/bbgid/BBG005P7Q8V5    0.279561
Name: PE_RATIO, dtype: float64

In [20]:
cer = excess_returns.cov()
cer

,/bbgid/BBG005P7Q8V5,/bbgid/BBG003P7K0B5,/bbgid/BBG000BB6RX0,/bbgid/BBG000MDCSN6,/bbgid/BBG000B9ZZG4,/bbgid/BBG000C2V541,/bbgid/BBG000B9Y5X2,/bbgid/BBG000F7RFH6,/bbgid/BBG000BB5BP5,/bbgid/BBG000D9DC22
/bbgid/BBG005P7Q8V5,0.000134,0.000135,0.000138,0.000135,0.000136,0.000137,0.000133,0.000138,0.000139,0.000140
/bbgid/BBG003P7K0B5,0.000135,0.000138,0.000140,0.000137,0.000138,0.000139,0.000135,0.000140,0.000141,0.000142
/bbgid/BBG000BB6RX0,0.000138,0.000140,0.000143,0.000140,0.000141,0.000142,0.000137,0.000143,0.000144,0.000145
/bbgid/BBG000MDCSN6,0.000135,0.000137,0.000140,0.000138,0.000138,0.000139,0.000135,0.000140,0.000141,0.000142
/bbgid/BBG000B9ZZG4,0.000136,0.000138,0.000141,0.000138,0.000138,0.000139,0.000135,0.000140,0.000141,0.000142
/bbgid/BBG000C2V541,0.000137,0.000139,0.000142,0.000139,0.000139,0.000140,0.000136,0.000141,0.000142,0.000144
/bbgid/BBG000B9Y5X2,0.000133,0.000135,0.000137,0.000135,0.000135,0.000136,0.000132,0.000137,0.000138,0.000139
/bbgid/BBG000F7RFH6,0.000138,0.000140,0.000143,0.000140,0.000140,0.000141,0.000137,0.000143,0.000143,0.000145
/bbgid/BBG000BB5BP5,0.000139,0.000141,0.000144,0.000141,0.000141,0.000142,0.000138,0.000143,0.000144,0.000146
/bbgid/BBG000D9DC22,0.000140,0.000142,0.000145,0.000142,0.000142,0.000144,0.000139,0.000145,0.000146,0.000147


In [21]:
factor_exposure = np.sum(E2P['PE_RATIO'] * weights)
factor_exposure

0.03740554976734908

In [22]:
asset_attributes = E2P['PE_RATIO']
asset_attributes

/bbgid/BBG000B9Y5X2    0.460858
/bbgid/BBG000B9ZZG4   -0.993148
/bbgid/BBG000BB5BP5   -1.401649
/bbgid/BBG000BB6RX0   -0.284416
/bbgid/BBG000C2V541   -0.831959
/bbgid/BBG000D9DC22   -0.182271
/bbgid/BBG000F7RFH6    0.136696
/bbgid/BBG000MDCSN6    0.875562
/bbgid/BBG003P7K0B5    0.166725
/bbgid/BBG005P7Q8V5    2.053602
Name: PE_RATIO, dtype: float64

In [11]:
ha = (cer**-1 * factor_exposure) / (asset_attributes.T * cer**-1 * factor_exposure)
ha = ha.iloc[0]
ha

/bbgid/BBG000B9Y5X2    2.169864
/bbgid/BBG000B9ZZG4   -1.006900
/bbgid/BBG000BB5BP5   -0.713445
/bbgid/BBG000BB6RX0   -3.515974
/bbgid/BBG000C2V541   -1.201982
/bbgid/BBG000D9DC22   -5.486331
/bbgid/BBG000F7RFH6    7.315507
/bbgid/BBG000MDCSN6    1.142124
/bbgid/BBG003P7K0B5    5.997887
/bbgid/BBG005P7Q8V5    0.486949
Name: /bbgid/BBG005P7Q8V5, dtype: float64

## Variance of characteristic portfolio

In [12]:
vcp = 1.0 / asset_attributes.T * cer**-1 * factor_exposure
vcp

,/bbgid/BBG000B9Y5X2,/bbgid/BBG000B9ZZG4,/bbgid/BBG000BB5BP5,/bbgid/BBG000BB6RX0,/bbgid/BBG000C2V541,/bbgid/BBG000D9DC22,/bbgid/BBG000F7RFH6,/bbgid/BBG000MDCSN6,/bbgid/BBG003P7K0B5,/bbgid/BBG005P7Q8V5
/bbgid/BBG005P7Q8V5,612.343531,-277.847432,-192.395785,-951.062223,-328.592985,-1467.146969,1983.159815,315.609418,1657.131203,136.134757
/bbgid/BBG003P7K0B5,603.415109,-273.594666,-189.685483,-937.255796,-323.747880,-1445.572114,1954.674267,310.786596,1630.940394,134.537157
/bbgid/BBG000BB6RX0,591.100276,-267.957681,-185.583444,-917.187469,-316.891238,-1414.565170,1913.972785,304.497534,1598.861349,131.718554
/bbgid/BBG000MDCSN6,603.174539,-273.645773,-189.718887,-937.381247,-323.715040,-1445.951228,1956.117518,310.636080,1632.102502,134.561388
/bbgid/BBG000B9ZZG4,602.619127,-273.193269,-189.311571,-935.676262,-323.145795,-1442.894987,1952.338016,310.395676,1629.745147,134.370462
/bbgid/BBG000C2V541,597.188487,-270.699143,-187.492422,-926.953620,-320.031219,-1429.185997,1934.829977,307.594219,1615.501537,133.120237
/bbgid/BBG000B9Y5X2,616.144640,-279.638233,-193.851159,-957.798603,-330.808524,-1477.744936,1998.649445,317.485253,1667.945621,137.418813
/bbgid/BBG000F7RFH6,592.822615,-268.718015,-186.132474,-919.892188,-317.904157,-1418.522701,1917.703047,305.396203,1602.611436,132.006984
/bbgid/BBG000BB5BP5,589.576833,-267.179297,-184.945709,-914.585421,-315.879172,-1410.044144,1908.560898,303.712825,1594.673615,131.316295
/bbgid/BBG000D9DC22,584.453585,-264.812760,-183.362807,-906.539109,-313.115498,-1397.419746,1891.466711,301.012650,1580.359952,130.219261


In [13]:
vcp2 = ha.T * cer * ha
vcp2

,/bbgid/BBG000B9Y5X2,/bbgid/BBG000B9ZZG4,/bbgid/BBG000BB5BP5,/bbgid/BBG000BB6RX0,/bbgid/BBG000C2V541,/bbgid/BBG000D9DC22,/bbgid/BBG000F7RFH6,/bbgid/BBG000MDCSN6,/bbgid/BBG003P7K0B5,/bbgid/BBG005P7Q8V5
/bbgid/BBG005P7Q8V5,0.000624,0.000137,0.000071,0.001709,0.000198,0.004210,0.007384,0.000177,0.004871,0.000032
/bbgid/BBG003P7K0B5,0.000633,0.000140,0.000072,0.001735,0.000201,0.004273,0.007492,0.000179,0.004949,0.000032
/bbgid/BBG000BB6RX0,0.000647,0.000143,0.000073,0.001773,0.000205,0.004367,0.007651,0.000183,0.005048,0.000033
/bbgid/BBG000MDCSN6,0.000634,0.000140,0.000072,0.001734,0.000201,0.004272,0.007486,0.000179,0.004945,0.000032
/bbgid/BBG000B9ZZG4,0.000634,0.000140,0.000072,0.001738,0.000201,0.004281,0.007501,0.000180,0.004952,0.000032
/bbgid/BBG000C2V541,0.000640,0.000141,0.000072,0.001754,0.000203,0.004322,0.007569,0.000181,0.004996,0.000032
/bbgid/BBG000B9Y5X2,0.000620,0.000137,0.000070,0.001697,0.000196,0.004180,0.007327,0.000176,0.004839,0.000031
/bbgid/BBG000F7RFH6,0.000645,0.000142,0.000073,0.001767,0.000204,0.004355,0.007636,0.000182,0.005036,0.000033
/bbgid/BBG000BB5BP5,0.000648,0.000143,0.000073,0.001778,0.000206,0.004381,0.007673,0.000183,0.005061,0.000033
/bbgid/BBG000D9DC22,0.000654,0.000144,0.000074,0.001793,0.000207,0.004420,0.007742,0.000185,0.005107,0.000033


## Beta with respect to Characteristic Portfolio

In [14]:
beta = cer * ha / vcp2
beta = beta.iloc[0]
beta

/bbgid/BBG000B9Y5X2    0.460858
/bbgid/BBG000B9ZZG4   -0.993148
/bbgid/BBG000BB5BP5   -1.401649
/bbgid/BBG000BB6RX0   -0.284416
/bbgid/BBG000C2V541   -0.831959
/bbgid/BBG000D9DC22   -0.182271
/bbgid/BBG000F7RFH6    0.136696
/bbgid/BBG000MDCSN6    0.875562
/bbgid/BBG003P7K0B5    0.166725
/bbgid/BBG005P7Q8V5    2.053602
Name: /bbgid/BBG005P7Q8V5, dtype: float64